# Fetch Codebase and Models

In [ ]:
import os
os.chdir('/content')
CODE_DIR = 'Latent_Attribute_Privacy'
!git clone https://github.com/jamelof23/Latent_Attribute_Privacy $CODE_DIR
os.chdir(f'./{CODE_DIR}/models/interfacegan_official')
!wget https://www.dropbox.com/s/qyv37eaobnow7fu/stylegan_ffhq.pth?dl=1 -O /content/Latent_Attribute_Privacy/models/interfacegan_official/models/pretrain/stylegan_ffhq.pth --quiet

# Define Utility Functions

In [ ]:
import os.path
import io
import IPython.display
import numpy as np
import cv2
import PIL.Image

import torch

# A dictionary containing information about the models available (e.g., pggan, stylegan).
from models.model_settings import MODEL_POOL

from models.stylegan_generator import StyleGANGenerator


def build_generator(model_name):
  """Builds the generator by model name."""
  gan_type = MODEL_POOL[model_name]['gan_type']
  if gan_type == 'stylegan':
    generator = StyleGANGenerator(model_name)
  return generator

import cv2
import numpy as np
from PIL import Image
import IPython.display as display
import io

def imshow(image, viz_size=256):

    # Ensure the input is a single image
    if image.ndim != 3:
        raise ValueError("Input must be a single image with 3 dimensions (height, width, channels).")

    # Resize the image if necessary
    height, width, channels = image.shape
    if height != viz_size or width != viz_size:
        image = cv2.resize(image, (viz_size, viz_size))

    # Convert image to a displayable format
    image = np.asarray(image, dtype=np.uint8)
    data = io.BytesIO()
    Image.fromarray(image).save(data, 'jpeg')
    im_data = data.getvalue()

    # Display the image
    return display.display(display.Image(im_data))

# Select a Model

In [ ]:
# Fixed parameters
model_name = "stylegan_ffhq"  # Always use 'stylegan_ffhq'
latent_space_type = "W"       # Always use latent space type 'W'

# Function to build and load the generator model
generator = build_generator(model_name)

# List of attributes for manipulation
ATTRS = ['age', 'gender']
# Dictionary to store attribute boundaries
boundaries = {}

# Loading Attribute Boundaries
for attr_name in ATTRS:
    boundary_name = f'{model_name}_{attr_name}'
    # Load the correct boundary file based on the latent space type
    boundary_path = f'boundaries/{boundary_name}_w_boundary.npy'
    boundaries[attr_name] = np.load(boundary_path)


# Upload initial latent code for a synthesized image

In [ ]:
import numpy as np
from google.colab import files

#num_samples = 1

# Upload the .npy file
print("[INFO] Please upload your latent vector (.npy file):")
uploaded = files.upload()

# Load the uploaded latent vector
npy_file_name = list(uploaded.keys())[0]
latent_codes = np.load(npy_file_name)

# Ensure that the latent vector is in the correct format (numpy.ndarray) and shape
if not isinstance(latent_codes, np.ndarray) or latent_codes.shape != (1, 512):
    raise ValueError(f"Latent codes must be a numpy.ndarray with shape (1, 512), but got {latent_codes.shape}")

# Use W space for StyleGAN synthesis
synthesis_kwargs = {'latent_space_type': 'W'}

# Generate the image using customized latent vector
images = generator.easy_synthesize(latent_codes, **synthesis_kwargs)['image']

# Extract the first image (assuming images has a batch dimension)
if images.ndim == 4:  # e.g., (batch_size, height, width, channels)
    images = images[0]

# Ensure the image is in the correct shape
if images.ndim != 3:
    raise ValueError(f"Generated image must have shape (height, width, channels), but got {images.shape}")

# Display the generated image
imshow(images)

# Privatizing attributes for synthesized image

In [ ]:
import numpy as np

# Differential Privacy Parameters
epsilon = 0.5  # @param {"type":"slider","min":0.01,"max":1,"step":0.01}
delta = 0.00001  # @param {type:"slider", min:1e-7, max:1e-3, step:1e-7}
clipping_threshold = 1  # @param {type:"slider", min:0, max:10, step:0.05}


# Dropdown Menu for Attribute Selection
attribute_selection = "Age"  # @param ["Age", "Gender"]

# Function to orthogonalize boundaries
def orthogonalize_boundaries(boundaries):

    """
    Orthogonalizes the attribute boundaries using Gram-Schmidt process.

    Args:
        boundaries (dict): Dictionary of attribute boundaries.

    Returns:
        dict: Dictionary of orthogonalized attribute boundaries.
    """

    orthogonal_boundaries = {}
    keys = list(boundaries.keys())
    for i, key_i in enumerate(keys):
        boundary_i = boundaries[key_i].copy()
        for j in range(i):
            key_j = keys[j]
            boundary_j = orthogonal_boundaries[key_j]
            projection = np.dot(boundary_i.flatten(), boundary_j.flatten()) / np.dot(boundary_j.flatten(), boundary_j.flatten())
            boundary_i -= projection * boundary_j
        orthogonal_boundaries[key_i] = boundary_i / np.linalg.norm(boundary_i)  # Normalize
    return orthogonal_boundaries

# Orthogonalize the boundaries
orthogonal_boundaries = orthogonalize_boundaries(boundaries)


# Assuming snsitivity
lambda_i = 1

# Function to add scalar noise
def add_scalar_noise(epsilon, delta, sensitivity):
    sigma = (sensitivity * np.sqrt(2 * np.log(1.25 / delta))) / epsilon
    return np.random.normal(0, sigma)



# Copy the original latent codes
new_codes = latent_codes.copy()

# Map attribute_selection to the list of attributes to modify
if attribute_selection == "Age":
    selected_attributes = ["age"]
elif attribute_selection == "Gender":
    selected_attributes = ["gender"]
else:
    selected_attributes = []


# Initialize the total update vector
delta_w_total = np.zeros_like(new_codes)

# Accumulate semantic shifts with noise
for attr_name in selected_attributes:
    # Retrieve the orthogonalized boundary for the attribute
    boundary = orthogonal_boundaries[attr_name]

    # Generate scalar noise for the attribute
    scalar_noise = add_scalar_noise(epsilon, delta, lambda_i)

    # Compute the semantic shift
    delta_w_total += scalar_noise * boundary

# Clip the total update to norm <= clipping_threshold
norm = np.linalg.norm(delta_w_total)
if norm > clipping_threshold:
    delta_w_total = (clipping_threshold / norm) * delta_w_total

# Apply the clipped update to the latent codes
new_codes += delta_w_total

# Generate the new images
new_images = generator.easy_synthesize(new_codes, **synthesis_kwargs)['image']

# Ensure only a single image is passed to imshow
if new_images.ndim == 4:  # If batch dimension exists
    new_images = new_images[0]

# Validate the image shape
if new_images.ndim != 3:
    raise ValueError(f"Generated image must have shape (height, width, channels), but got {new_images.shape}")

# Display the new image
imshow(new_images)
